# Creating PointNet Model in Tensorflow 2
Explain what is Pointnet Model?

The Architecture main model is given below:
![](https://i.imgur.com/ATtugK4.png)

In [ ]:
import os                               # Handling Data
import glob                             
import numpy as np                      # Fast Matrix Processing
import tensorflow as tf                 # Tensorflow 2
from tensorflow import keras            # Keras
from tensorflow.keras import layers     # Layers
from matplotlib import pyplot as plt    # Plotting
tf.random.set_seed(1234)  
tf.__version__              

'2.7.0'

In [ ]:
def convolutionalLayersWithBatchNormalisation(modelInternals, filters):
    modelInternals = layers.Conv1D(filters, kernel_size=1, padding="valid")(modelInternals)
    modelInternals = layers.BatchNormalization(momentum=0.0)(modelInternals)
    return layers.Activation("relu")(modelInternals)

def denseLayersWithBatchNormalisation(modelInternals, filters):
    modelInternals = layers.Dense(filters)(modelInternals)
    modelInternals = layers.BatchNormalization(momentum=0.0)(modelInternals)
    return layers.Activation("relu")(modelInternals)

class OrthogonalRegularizer(keras.regularizers.Regularizer):
    """
        Attaching orthognal regulariser to model
    """
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = tf.eye(num_features)

    def __call__(self, modelInternals):
        modelInternals = tf.reshape(modelInternals, (-1, self.num_features, self.num_features))
        xxt = tf.tensordot(modelInternals, modelInternals, axes=(2, 2))
        xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
        return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))

#### T-net layers

In [ ]:
def tnet(inputs, num_features):
    # Initalise bias as the indentity matrix
    bias = keras.initializers.Constant(np.eye(num_features).flatten())
    # regulariser
    reg = OrthogonalRegularizer(num_features)
    # add conv layers
    modelInternals = convolutionalLayersWithBatchNormalisation(inputs, 32)
    modelInternals = convolutionalLayersWithBatchNormalisation(modelInternals, 64)
    modelInternals = convolutionalLayersWithBatchNormalisation(modelInternals, 512)
    # add maxpooling 1D
    modelInternals = layers.GlobalMaxPooling1D()(modelInternals)
    # add dense layers
    modelInternals = denseLayersWithBatchNormalisation(modelInternals, 256)
    modelInternals = denseLayersWithBatchNormalisation(modelInternals, 128)
    modelInternals = layers.Dense(
        num_features * num_features,
        kernel_initializer="zeros",
        bias_initializer=bias,
        activity_regularizer=reg,
    )(modelInternals)
    feat_T = layers.Reshape((num_features, num_features))(modelInternals)
    # Apply affine transformation to input features
    return layers.Dot(axes=(2, 1))([inputs, feat_T])

In [ ]:
def getPointNet3D(NUM_CLASSES,NUM_POINTS):
    inputs = keras.Input(shape=(NUM_POINTS, 3))
    model = tnet(inputs, 3)
    model = convolutionalLayersWithBatchNormalisation(model,32)
    model = convolutionalLayersWithBatchNormalisation(model,32)
    model = tnet(model,32)
    model = convolutionalLayersWithBatchNormalisation(model,32)
    model = convolutionalLayersWithBatchNormalisation(model,64)
    model = convolutionalLayersWithBatchNormalisation(model,512)
    model = layers.GlobalMaxPooling1D()(model)
    model = denseLayersWithBatchNormalisation(model,256)
    model = layers.Dropout(0.3)(model)
    model = denseLayersWithBatchNormalisation(model,128)
    model = layers.Dropout(0.3)(model)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax")(model)
    model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet")
    model.summary()
    return model

## ModelNet40 Dataset
It is a dataset ....


In [ ]:
!wget https://shapenet.cs.stanford.edu/media/modelnet40_ply_hdf5_2048.zip --no-check-certificate

--2021-11-30 20:56:45--  https://shapenet.cs.stanford.edu/media/modelnet40_ply_hdf5_2048.zip
Resolving shapenet.cs.stanford.edu (shapenet.cs.stanford.edu)... 171.67.77.19
Connecting to shapenet.cs.stanford.edu (shapenet.cs.stanford.edu)|171.67.77.19|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 435212151 (415M) [application/zip]
Saving to: ‘modelnet40_ply_hdf5_2048.zip’

modelnet40_ply_hdf5 100%[===================>] 415.05M  11.1MB/s    in 37s     

2021-11-30 20:57:23 (11.1 MB/s) - ‘modelnet40_ply_hdf5_2048.zip’ saved [435212151/435212151]



In [ ]:
!unzip /content/modelnet40_ply_hdf5_2048.zip

Archive:  /content/modelnet40_ply_hdf5_2048.zip
   creating: modelnet40_ply_hdf5_2048/
  inflating: modelnet40_ply_hdf5_2048/ply_data_train_2_id2file.json  
  inflating: modelnet40_ply_hdf5_2048/ply_data_train2.h5  
  inflating: modelnet40_ply_hdf5_2048/ply_data_train4.h5  
  inflating: modelnet40_ply_hdf5_2048/ply_data_train1.h5  
  inflating: modelnet40_ply_hdf5_2048/train_files.txt  
  inflating: modelnet40_ply_hdf5_2048/ply_data_train_4_id2file.json  
  inflating: modelnet40_ply_hdf5_2048/ply_data_test1.h5  
  inflating: modelnet40_ply_hdf5_2048/ply_data_test0.h5  
  inflating: modelnet40_ply_hdf5_2048/ply_data_test_1_id2file.json  
  inflating: modelnet40_ply_hdf5_2048/ply_data_train_1_id2file.json  
  inflating: modelnet40_ply_hdf5_2048/ply_data_train_0_id2file.json  
  inflating: modelnet40_ply_hdf5_2048/test_files.txt  
  inflating: modelnet40_ply_hdf5_2048/ply_data_train0.h5  
  inflating: modelnet40_ply_hdf5_2048/ply_data_test_0_id2file.json  
  inflating: modelnet40_ply_hdf5

In [ ]:
import h5py
# colleting train data from 4 hdfs files
fp = h5py.File("/content/modelnet40_ply_hdf5_2048/ply_data_train0.h5",'r')  
fp1 = h5py.File("/content/modelnet40_ply_hdf5_2048/ply_data_train1.h5",'r')
fp2 = h5py.File("/content/modelnet40_ply_hdf5_2048/ply_data_train2.h5",'r')
fp3 = h5py.File("/content/modelnet40_ply_hdf5_2048/ply_data_train3.h5",'r')
fp4 = h5py.File("/content/modelnet40_ply_hdf5_2048/ply_data_train4.h5",'r')
trainX = tf.convert_to_tensor(np.concatenate((np.array(fp["data"]),np.array(fp1["data"]),np.array(fp2["data"]),np.array(fp3["data"]),np.array(fp4["data"]))),dtype=tf.float32)
trainy = tf.convert_to_tensor(np.concatenate((np.array(fp["label"]),np.array(fp1["label"]),np.array(fp2["label"]),np.array(fp3["label"]),np.array(fp4["label"]))))
train_dataset = tf.data.Dataset.from_tensor_slices((trainX,trainy))
fp = h5py.File("/content/modelnet40_ply_hdf5_2048/ply_data_test0.h5",'r')  
fp1 = h5py.File("/content/modelnet40_ply_hdf5_2048/ply_data_test1.h5",'r')  
# print(np.array(fp["data"]).shape)
testX = tf.convert_to_tensor(np.concatenate((np.array(fp["data"]),np.array(fp1["data"]))),dtype=tf.float32)
testy = tf.convert_to_tensor(np.concatenate((np.array(fp["label"]),np.array(fp1["label"]))))
test_dataset = tf.data.Dataset.from_tensor_slices((testX,testy))
print(len(train_dataset))
print(len(test_dataset))

9840
2468


In [ ]:
NUM_POINTS = 2048
NUM_CLASSES = 40
BATCH_SIZE = 32
train_dataset = train_dataset.shuffle(len(train_dataset)).batch(BATCH_SIZE)
test_dataset = test_dataset.shuffle(len(test_dataset)).batch(BATCH_SIZE)

In [ ]:
pointNet = getPointNet3D(NUM_CLASSES,NUM_POINTS)
pointNet.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["sparse_categorical_accuracy"],
)
history = pointNet.fit(train_dataset, epochs=30, validation_data=test_dataset)

Model: "pointnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2048, 3)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 2048, 32)     128         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 2048, 32)    128         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 2048, 32)     0           ['batch_normalization[0][0

In [ ]:
# Hyperparameters

exp_num = 1

def get_dataset(file_name = "/content/modelnet40_ply_hdf5_2048/ply_data_test0.h5"):
  global exp_num
  if exp_num == 1:
    fp = h5py.File(file_name,'r')  
    X = np.array(fp["data"])
    y = np.array(fp["label"])
    return X,y

def get_epsilon():
  return 1e-6

def get_TauUpdate():
  return 1.0, 1

def get_lr():
  global exp_num
  if exp_num == 1:
    return 1

def get_budget():
  global exp_num
  if exp_num == 1:
    return 0.05

def get_epochs():
  global exp_num
  if exp_num == 1:
    return 100 # Change Krna h

In [ ]:
# Use GPU for training
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("GPUs Available: ", len(physical_devices))

GPUs Available:  1


In [ ]:
def accuracy(y,y_hat):
  for i in range(len(y)):
    if y_[i]==y__[i]:
      ans+=1
  return ans/len(y)

In [ ]:
from sklearn.metrics import precision_score, recall_score
# Get ModelNet40 Test Dataset
P, y = get_dataset()

y = tf.keras.utils.to_categorical(y)
r, s = 1.004, 1
epochs = 100
Budget = get_budget()
size = int(tf.shape(P)[1])
init_value = np.random.randn(size, 1).astype(np.float32)

# Creating Our Trainable Parameters
T = tf.Variable(trainable = False, initial_value = 1.0)
e = tf.Variable(trainable = True, initial_value = init_value)
Loss = tf.keras.losses.CategoricalCrossentropy()
k = 10
tolerance = k
best = float("inf")
lr = 0.1
for epoch in range(1, epochs+1):
  # Creating Computational Graph of The Entire Forward Pass
  with tf.GradientTape() as tape1, tf.GradientTape() as tape2:
    tape1.watch(e)
    tape2.watch(e)
    X0 = tf.math.sigmoid(tf.math.multiply(T, e))
    X1 = tf.grad_pass_through(tf.math.round)(X0)
    X = tf.math.multiply(P, X1)
    y_hat = pointNet(X)
    loss = -Loss(y, y_hat)
    cost_overrun = (tf.math.reduce_mean(X1)/size) - Budget
  # Gradients
  grad_o = tape1.gradient(loss, e)
  grad_b = tape2.gradient(cost_overrun, e)
  y_ = tf.math.argmax(y,axis=1)
  y__ = tf.math.argmax(y_hat,axis=1)
  ans = 0
  accuracy(y_,y__)
  print("Epochs: ",epoch,"Accuracy: ",ans/len(y),"Loss: ",-loss.numpy().tolist())
  # If cost_overrun is less than 0 then set beta to 0
  if cost_overrun <= 0:
    Beta = 0
  else:
    # Else calculate the terms to update the beta
    ob = tf.reduce_sum(tf.multiply(grad_o, grad_b))
    bb = tf.reduce_sum(tf.multiply(grad_b, grad_b))
    oo = tf.reduce_sum(tf.multiply(grad_o, grad_o))
    t1 = ob / (cost_overrun * bb)
    t2 = oo / (cost_overrun * bo)
    if bo < 0:
        Beta = 0
    elif bo > 0 and t1 < t2:
        Beta = (t1 + t2) / 2
    else:
        Beta = t1 + (1e-6)
    # Early Stopping
    if cost_overrun > 0 and int(loss) < best:
      tolerance -= 1
    else:
      best = int(loss)
      tolerance = k
    if tolerance <= 0:
      break

  delL = grad_o - Beta*cost_overrun*grad_b + get_epsilon()
  e = e + tf.math.scalar_mul(lr, delL) # only update a portion
  T = tf.math.scalar_mul(pow(r, epoch//s), T)

Epochs:  1 Accuracy:  0.759493670886076 Loss:  1.0003095865249634
Epochs:  2 Accuracy:  0.7848101265822784 Loss:  0.9718227982521057
Epochs:  3 Accuracy:  0.7974683544303798 Loss:  0.9564444422721863
Epochs:  4 Accuracy:  0.7974683544303798 Loss:  0.928037703037262
Epochs:  5 Accuracy:  0.7974683544303798 Loss:  0.8966020941734314
Epochs:  6 Accuracy:  0.7974683544303798 Loss:  0.8868858218193054
Epochs:  7 Accuracy:  0.7974683544303798 Loss:  0.8731716275215149
Epochs:  8 Accuracy:  0.7974683544303798 Loss:  0.861458957195282
Epochs:  9 Accuracy:  0.810126582278481 Loss:  0.8698155879974365
Epochs:  10 Accuracy:  0.810126582278481 Loss:  0.8560367822647095
Epochs:  11 Accuracy:  0.8227848101265823 Loss:  0.8224151134490967
Early Stopping...
Breaked
Epochs:  11 Accuracy:  0.8227848101265823 Loss:  0.8224151134490967
